# 动机
不同病人之间的检测数据可能呈现不同的特征，而同一个病人的监测数据可能具有某种共性。因此，按入院的患者来做分组，可能是另一种可行的数据稠密化方案。

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib

# Load data from a pickle file
try:
    data : pd = pd.read_pickle('../data/data_remove_dup.pkl')
except:
    print("未在路径中找到目标数据表，先运行其他脚本")
    raise
data = data.iloc[:, 1:].drop(columns=["病床号"])
data = data.groupby("住院号码").mean(numeric_only=True).reset_index()
import matplotlib.pyplot as plt

# Select the specified columns
groups = [['C肽（餐后2小时）', '胰岛素（餐后2小时）', '葡萄糖(餐后2小时)'],
          ['促甲状腺素', '反三碘甲状腺原氨酸', '总三碘甲状腺原氨酸', '总四碘甲状腺原氨酸', '游离三碘甲状腺原氨酸',
           '游离甲状腺素', '甲状腺球蛋白', '甲状腺过氧化物酶抗体'],
          ['β-胶原特殊序列', '促甲状腺素受体抗体', '总I型胶原氨基端延长肽', '骨钙素(N-MID)'],
          ['25-羟基维生素D', '甲状旁腺激素', '降钙素'],
          ['C肽', '胰岛素'],
          ['促卵泡成熟素', '促黄体生成素', '叶酸', '孕酮', '泌乳素', '睾酮', '硫酸去氢表雄酮', '雌二醇'],
          ['干扰素γ', '白介素10', '白介素17A', '白介素1β', '白介素2', '白介素2受体', '白介素4', '白介素5', '白介素6',
           '白介素8', '肿瘤坏死因子α'],
          ['低密度脂蛋白', '总胆固醇', '甘油三酯', '磷', '糖化白蛋白', '葡萄糖', '镁', '高密度脂蛋白'],
          ['尿素', '尿酸', '氯', '肌酐', '钙', '钠', '钾'],
          ['γ-谷氨酰转肽酶', '天门冬氨酸转氨酶', '总胆红素', '直接胆红素', '碱性磷酸酶']]
# 对所有 groups 中的列分别做PCA，并绘图展示
for idx, group in enumerate(groups):
    # 过滤出当前 group 中实际存在于 data 的列
    valid_columns = [col for col in group if col in data.columns]
    if not valid_columns:
        print(f"Group {idx+1} 没有有效的列，跳过。")
        continue
    group_data = data[valid_columns].dropna()
    if group_data.shape[0] < 3 or group_data.shape[1] < 2:
        print(f"Group {idx+1} 数据量不足，跳过。")
        continue
    # 归一化
    scaler = MinMaxScaler()
    # scaler = StandardScaler()
    group_normalized = scaler.fit_transform(group_data)
    # PCA
    group_pca = PCA()
    group_pca_result = group_pca.fit_transform(group_normalized)
    # 解释方差
    print(f"Group {idx+1} ({valid_columns}) explained variance ratio:", group_pca.explained_variance_ratio_)
    # 绘图（弹出窗口，支持交互旋转）
    from mpl_toolkits.mplot3d import Axes3D  # 确保3D支持
    matplotlib.use('tkagg')  # 使用tkagg后端弹窗

    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection="3d")
    ax.scatter(
        group_pca_result[:, 0],
        group_pca_result[:, 1],
        group_pca_result[:, 2] if group_pca_result.shape[1] > 2 else [0]*group_pca_result.shape[0],
        alpha=0.7
    )
    ax.set_title(f"PCA Result (3D) - Group {idx+1}")
    ax.set_xlabel("PC1")
    ax.set_ylabel("PC2")
    ax.set_zlabel("PC3")
    plt.show(block=True)
